# Setup

In [0]:
# Get CIFAR-10 dataset in .png format
wget http://pjreddie.com/media/files/cifar.tgz
tar xzf cifar.tgz

# Clone ESRGAN repo here
!git clone https://github.com/xinntao/ESRGAN.git
  
# Folders required for loading and storing images 
!mkdir LR 
!mkdir results

# Download Pretrained Models from Drive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Download a file based on its file ID.
#
# A file ID looks like: laggVyWshwcyP6kEI-y_W3P8D26sz
file_id = '1mSJ6Z40weL-dnPvi390xDd3uZBCFMeqr'
X = drive.CreateFile({'id': file_id})
X = X.GetContentFile('ESRGAN/models/RRDB_PSNR.pth')

file_id = '1MJFgqXJrMkPdKtiuy7C6xfsU1QIbXEb-'
Y = drive.CreateFile({'id': file_id})
Y = Y.GetContentFile('ESRGAN/models/RRDB_ESRGAN.pth')